In [6]:
import sys
import os
from pathlib import Path

# Add parent directory to Python path
notebook_dir = Path(os.getcwd())
project_dir = notebook_dir.parent
sys.path.append(str(project_dir))


# notebooks/02_model_training.ipynb

import pandas as pd
import numpy as np
from src.data_processor import DataProcessor
from src.models.expenditure_predictor import ExpenditurePredictor
import plotly.graph_objects as go

# Prepare data
processor = DataProcessor()
df = processor.load_household_data('../data/sample_household_data.csv')

# Train separate models for each household
household_models = {}
household_metrics = {}

for household in df['HouseholdID'].unique():
    print(f"\nTraining model for {household}")
    
    # Filter data for current household
    household_df = df[df['HouseholdID'] == household].copy()
    X, y = processor.prepare_training_data(household_df)
    
    # Split data
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    
    # Train model
    predictor = ExpenditurePredictor()
    history = predictor.train(X_train, y_train)
    
    # Evaluate model
    y_pred = predictor.predict(X_test)
    mse = np.mean((y_test - y_pred.flatten()) ** 2)
    
    # Store model and metrics
    household_models[household] = predictor
    household_metrics[household] = {
        'mse': mse,
        'history': history
    }
    
    print(f"MSE for {household}: {mse:.4f}")

# Visualize training history for each household
fig = go.Figure()
for household, metrics in household_metrics.items():
    history = metrics['history']
    fig.add_trace(go.Scatter(
        y=history.history['loss'],
        name=f'{household} Training Loss',
        mode='lines'
    ))
    fig.add_trace(go.Scatter(
        y=history.history['val_loss'],
        name=f'{household} Validation Loss',
        mode='lines',
        line=dict(dash='dash')
    ))

fig.update_layout(
    title='Training History by Household',
    xaxis_title='Epoch',
    yaxis_title='Loss',
    hovermode='x unified'
)
fig.show()

# Compare predictions vs actual for each household
for household in household_models:
    household_df = df[df['HouseholdID'] == household].copy()
    X, y = processor.prepare_training_data(household_df)
    
    # Get predictions
    y_pred = household_models[household].predict(X)
    
    # Plot actual vs predicted
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        y=y,
        name='Actual',
        mode='lines'
    ))
    fig.add_trace(go.Scatter(
        y=y_pred.flatten(),
        name='Predicted',
        mode='lines'
    ))
    
    fig.update_layout(
        title=f'Actual vs Predicted Expenditure - {household}',
        xaxis_title='Time',
        yaxis_title='Expenditure ($)',
        hovermode='x unified'
    )
    fig.show()



Training model for H1
Epoch 1/100


c:\Users\utkar\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 326ms/step - loss: 0.3319 - val_loss: 0.1039
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.2594 - val_loss: 0.0724
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.1977 - val_loss: 0.0444
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.1207 - val_loss: 0.0301
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0679 - val_loss: 0.0469
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0443 - val_loss: 0.1018
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.0496 - val_loss: 0.1237
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - loss: 0.0590 - val_loss: 0.0927
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 0.0473 - val_loss: 0.0607
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.0348 - val_loss: 0.0487
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.0347 - val_loss: 0.0459
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.0323 - val_loss: 0.0465


c:\Users\utkar\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 287ms/step - loss: 0.4006 - val_loss: 0.1529
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.3192 - val_loss: 0.1150
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.2625 - val_loss: 0.0859
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.2111 - val_loss: 0.0617
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.1570 - val_loss: 0.0447
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.1126 - val_loss: 0.0420
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.0662 - val_loss: 0.0644
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.0455 - val_loss: 0.1053
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.0487 - val_loss: 0.1252
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0465 - val_loss: 0.1277
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0493 - val_loss: 0.1095
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.0403 - val_loss: 0.0841


...

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step 


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step 


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step 


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step 


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step 
